In [1]:
%pip install langchain-community langchain-pinecone langchain-openai pinecone python-dotenv  -q
# from google.colab import userdata
import os
import ast

Note: you may need to restart the kernel to use updated packages.


In [2]:
# os.environ['LANGSMITH_TRACING'] = userdata.get('LANGSMITH_TRACING')
# os.environ['LANGSMITH_ENDPOINT'] = userdata.get('LANGSMITH_ENDPOINT')
# os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
# os.environ['LANGSMITH_PROJECT'] = userdata.get('LANGSMITH_PROJECT')
# os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
# os.environ['PINECONE_PJ_KEY'] = userdata.get('PINECONE_PJ_KEY')


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# # 구글 드라이브 연동
# from google.colab import drive
# drive.mount('/content/drive')

# BASE_PATH = '/content/drive/Othercomputers/내 노트북/Workspaces/deep_learning_multimodal_workspace/HanSeongGyu/05_multimodal_rag'

In [15]:
%%time

# 사용 방법 예시 1

from inferer import *

names   = ["food1.jpg", "food2.jpg", "food3.jpg"]       # 이미지 파일 경로
images  = [Inferer.to_pil_image(name) for name in names]    # 파일경로를 받아 PIL.Image 로 변환

inferer = OpenAIInferer("gpt-4o-mini", 0.0)                # 음식 이미지를 추론할 모델 선언
results = inferer(images, names)                            # 이미지 목록과 이름 목록을 받아서 추론 시작
results                                                     # 결과

CPU times: total: 93.8 ms
Wall time: 2.88 s


{'food1.jpg': '[("짜장면", "춘장, 돼지고기, 양파, 면, 카라멜")]',
 'food2.jpg': '[("핫도그", "소시지, 밀가루, 옥수수 가루, 계란, 우유, 케첩, 머스터드")]',
 'food3.jpg': '[("흑미 피자", "흑미 도우, 치즈, 올리브, 토마토, 양상추, 고기, 양파")]'}

In [16]:
import os
import ast
from typing import List, Tuple
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
import openai
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# --- 1. Pinecone & Index 초기화 ---
PINECONE_PJ_KEY = os.environ["PINECONE_PJ_KEY"]
INDEX_NAME = "food-index"
EMBED_MODEL = "text-embedding-3-small"

pc = Pinecone(api_key=PINECONE_PJ_KEY)
index = pc.Index(INDEX_NAME)
embeddings = OpenAIEmbeddings(model=EMBED_MODEL)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

# --- 2. Prediction 파싱 함수 ---
def parse_prediction(pred_str: str) -> Tuple[str, str]:
    parsed = ast.literal_eval(pred_str)
    menu_name, ingredients = parsed[0]
    return menu_name.strip(), ingredients.strip()

# --- 3. Pinecone 검색 ---
def search_menu(menu_name: str, k: int = 3) -> List[Tuple]:
    return vector_store.similarity_search_with_score(query=menu_name, k=k)

# --- 4. Pinecone 결과 → 컨텍스트 형식 변환 ---
def build_context(matches: List[Tuple]) -> str:
    lines = []
    for doc, score in matches:
        meta = doc.metadata or {}
        name = meta.get("RCP_NM", "알 수 없는 메뉴")
        kcal = meta.get("INFO_ENG", "칼로리 정보 없음")
        lines.append(f"- 메뉴명: {name}, 칼로리: {kcal} (유사도: {score:.2f})")
    return "\n".join(lines)

# --- 5. LLM 사용한 칼로리 추정 ---
def ask_llm_calorie(menu_name: str) -> str:
    prompt = (
        f"다음 음식의 대표적인 1인분 칼로리(kcal) 숫자만 알려주세요 **반드시 숫자만 반환!!**:\n"
        f"‘{menu_name}’"
    )
    resp = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    return resp.choices[0].message.content.strip()

# --- 6. 메뉴명 기반 컨텍스트 생성 + 칼로리 반환 개선 버전 ---
def get_menu_context_with_threshold(
    menu_name: str,
    k: int = 1,
    threshold: float = 0.4
) -> Tuple[str, str]:
    matches = search_menu(menu_name, k)
    
    if not matches or matches[0][1] < threshold:
        # 유사도 낮을 경우 LLM으로 fallback
        calorie = ask_llm_calorie(menu_name)
        context = f"- 메뉴명: {menu_name}, 칼로리: {calorie}"
        return context, calorie

    # 유사한 문서가 충분함 → 문서에서 kcal 추출
    context = build_context(matches)
    # 가장 첫 번째 문서 정보 사용
    doc, _ = matches[0]
    calorie = doc.metadata.get("INFO_ENG")

    # 칼로리 정보가 누락되어 있을 경우 fallback
    if not calorie or not str(calorie).isdigit():
        calorie = ask_llm_calorie(menu_name)

    return context, calorie

# --- 7. LLM Chain 구성 ---
llm = ChatOpenAI(model='gpt-4.1', temperature=0.3)
output_parser = StrOutputParser()
prompt = PromptTemplate.from_template("""
[System Instruction]
- 당신은 영양&운동 전문 PT "G-PT"입니다.
- 음식 이미지 분류 결과로 얻은 menu_name과 텍스트 정보를 바탕으로
  섭취 열량 및 남은 권장 열량에 맞는 식단과 운동을 추천합니다.
- Pinecone RAG에서 유사도 0.4 미만일 경우 LLM에게 fallback합니다.
- 1일 권장 섭취량은 사용자의 입력 text 바탕으로 계산을 해서 사용합니다.
- 음식 이미지가 여러 개일 경우 각 메뉴를 따로 분석합니다.

[Input Data]
{rag_context}
{text}

[Output Indicator]
" 드신 메뉴인 '메뉴명 :'{menu_name}, 은 '칼로리:' {calorie}kcal" 입니다.
오늘의 1일 권장 섭취량 (권장 kcal) 중 ((권장 kcal) - {calorie}- (이전 식사의 kcal)) 남았습니다.

방금 드신 {menu_name}의 열량인 {calorie}kcal를 빼시려면
xxx(운동명) xx분 (x)kcal,
yyy(운동명) yy분 (y)kcal,
zzz(운동명) zz분 (z)kcal 을 하셔야합니다.

남은 섭취량 ((권장 kcal) - {calorie} - (이전 식사의 kcal))의 추천 식단은
oooo(음식명) oo(양) o(kcal),
pppp(음식명) pp(양) p(kcal),
qqqq(음식명) qq(양) q(kcal)입니다.
""")
chain = prompt | llm | output_parser

# --- 8. Prompt chain 호출 ---
def analyze_meal(rag_context: str, text: str, menu_name: str, calorie: str) -> str:
    return chain.invoke({
        'rag_context': rag_context,
        'text': text,
        'menu_name': menu_name,
        'calorie': calorie,
    })

# --- 9. 실행부 ---
if __name__ == '__main__':
    # 테스트용 예시 results 딕셔너리

    parsed_items = []
    for img_path, pred_str in results.items():
        try:
            menu_name, ingredients = parse_prediction(pred_str)
            parsed_items.append((img_path, menu_name, ingredients))
        except Exception as e:
            print(f"[ERROR] {img_path}: {e}")

    for img_path, menu_name, ingredients in parsed_items:
        print(f"\n[INFO] 처리 중: {img_path} - {menu_name}")
        try:
            rag_ctx = get_menu_context_with_threshold(menu_name=menu_name)
            calorie = ask_llm_calorie(menu_name)
            text = "사용자가 입력한 신체 정보 및 조건 텍스트"  # 예를 들면 키, 몸무게 등
            result = analyze_meal(rag_ctx, text, menu_name, calorie)
            print(f"[RESULT] {menu_name}\n{result}\n")
        except Exception as e:
            print(f"[ERROR] {img_path}: {e}")




[INFO] 처리 중: food1.jpg - 짜장면
[RESULT] 짜장면
드신 메뉴인 '메뉴명 : 짜장면, 은 칼로리: 600kcal' 입니다.

오늘의 1일 권장 섭취량(권장 kcal)은 사용자의 신체 정보가 입력되지 않아, 평균 성인 남성 기준(2,500kcal)으로 계산하겠습니다.  
남은 권장 섭취량은 2,500 - 600 = 1,900kcal 입니다. (이전 식사의 kcal 정보가 없으므로, 오늘 첫 식사로 가정합니다.)

방금 드신 짜장면의 열량인 600kcal를 빼시려면  
- 빠르게 걷기 90분 (약 600kcal)  
- 자전거 타기(중간 강도) 70분 (약 600kcal)  
- 줄넘기(중간 강도) 50분 (약 600kcal)  
정도의 운동이 필요합니다.

남은 섭취량 1,900kcal의 추천 식단은  
- 닭가슴살 샐러드 200g (약 220kcal)  
- 고구마 150g (약 180kcal)  
- 현미밥 1공기(210g) (약 300kcal)  
- 두부구이 100g (약 120kcal)  
- 바나나 1개(100g) (약 90kcal)  
- 연어구이 100g (약 250kcal)  
- 야채스틱(오이, 당근 등) 100g (약 40kcal)  
등으로 구성하실 수 있습니다.

더 정확한 권장 섭취량과 맞춤 식단/운동을 원하시면 신체 정보(성별, 나이, 키, 체중, 목표 등)를 입력해 주세요!


[INFO] 처리 중: food2.jpg - 핫도그
[RESULT] 핫도그
드신 메뉴인 '핫도그'는 '칼로리: 250kcal' 입니다.

오늘의 1일 권장 섭취량을 계산하기 위해 신체 정보 및 조건이 필요합니다. 입력하신 신체 정보가 부족하여, 평균적인 성인 남성(30세, 170cm, 70kg, 활동량 보통 기준) 기준으로 1일 권장 섭취량을 약 2,400kcal로 가정하겠습니다.

오늘의 1일 권장 섭취량(2,400kcal) 중 2,150kcal(=2,400 - 250) 남았습니다.

방금 드신 핫도그의 열량인 250kcal를 빼시려